In [138]:
# Value Creation Bridge aligned to Achleitner et al. 2010; Achleitner, Braun & Puche 2015; Söffge & Braun 2017)

# === Core Value-Bridge Items: times_money and tm_unlevered (exit-row div/cap injection) ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + checks ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

def num(s): return pd.to_numeric(s, errors="coerce")

# Order within deal: 1 = entry (earliest), 2 = exit (latest)
df["_ref_dt"] = pd.to_datetime(df["reference_date"], errors="coerce")
df["_rank"]   = df.groupby("deal_id")["_ref_dt"].rank(method="first", ascending=True)

eq  = num(df["equity"])

# Entry / Exit equity
equity_entry = (df.loc[df["_rank"] == 1, ["deal_id"]]
                  .assign(equity_entry=eq[df["_rank"] == 1].values)
                  .groupby("deal_id")["equity_entry"].first())
equity_exit  = (df.loc[df["_rank"] == 2, ["deal_id"]]
                  .assign(equity_exit=eq[df["_rank"] == 2].values)
                  .groupby("deal_id")["equity_exit"].first())

# ---- deal-level inputs (exit-row dividends & capital injections) ----
div_exit = (df.loc[df["_rank"] == 2, ["deal_id"]]
              .assign(div_exit=num(df["dividends"])[df["_rank"] == 2].values)
              .groupby("deal_id")["div_exit"].first())

cap_exit = (df.loc[df["_rank"] == 2, ["deal_id"]]
              .assign(cap_exit=num(df["capital_injections"])[df["_rank"] == 2].values)
              .groupby("deal_id")["cap_exit"].first())

div_exit = div_exit.fillna(0.0)
cap_exit = cap_exit.fillna(0.0)

# ---- dtype & presence guards (drop-in right after div_exit/cap_exit fillna) ----
# Ensure numeric dtype and no missing equity at entry/exit (these are the two rows you rely on)
assert equity_entry.notna().all(), "Equity missing on entry rows."
assert equity_exit.notna().all(),  "Equity missing on exit rows."

# Make sure dividends / cap injections are numeric after fill
div_exit  = pd.to_numeric(div_exit, errors="coerce").fillna(0.0)
cap_exit  = pd.to_numeric(cap_exit, errors="coerce").fillna(0.0)


# ---- average D/E as entry/exit two-point mean; clip at 0 for unlevering ----
der_series = num(df["de_ratio"])
der_entry  = df.loc[df["_rank"] == 1, ["deal_id"]].assign(der=der_series[df["_rank"] == 1].values) \
                 .groupby("deal_id")["der"].first()
der_exit   = df.loc[df["_rank"] == 2, ["deal_id"]].assign(der=der_series[df["_rank"] == 2].values) \
                 .groupby("deal_id")["der"].first()
der_avg    = pd.concat([der_entry, der_exit], axis=1).mean(axis=1).clip(lower=0.0)

# ---- cost of debt: take the (deal-level) compounded total, not a mean ----
cod_spread = df.groupby("deal_id")["cost_of_debt"].agg(lambda s: float(s.max() - s.min()))
assert (cod_spread.abs() < 1e-12).all(), "cost_of_debt differs across rows; confirm intent."

cod_avg = df.groupby("deal_id")["cost_of_debt"].first()

# ---- formulas (deal level) ----
d_equity = equity_exit - equity_entry
net_capital_gain = d_equity + div_exit + cap_exit
invested_capital = equity_entry - cap_exit

# ---- invested capital guard: require strictly positive IC ----
valid_ic = invested_capital > 0
if not valid_ic.all():
    bad = (~valid_ic).sum()
    print(f"Dropping {bad} deal(s) with invested_capital <= 0.")
    # Hard drop of invalid deals from all deal-level vectors:
    equity_entry      = equity_entry[valid_ic]
    equity_exit       = equity_exit[valid_ic]
    div_exit          = div_exit[valid_ic]
    cap_exit          = cap_exit[valid_ic]
    d_equity          = d_equity[valid_ic]
    net_capital_gain  = net_capital_gain[valid_ic]
    invested_capital  = invested_capital[valid_ic]
    der_avg           = der_avg[valid_ic]
    cod_avg           = cod_avg[valid_ic]

# Remove invalid deals from the row-level frame as well
survivor_deals = set(invested_capital.index)  # index currently is deal_id for valid deals
df = df[df["deal_id"].isin(survivor_deals)].copy()


with np.errstate(divide="ignore", invalid="ignore"):
    times_money = (net_capital_gain / invested_capital).where(invested_capital != 0)

with np.errstate(divide="ignore", invalid="ignore"):
    tm_unlevered = (times_money + cod_avg * der_avg) / (1 + der_avg)

leverage_effect = times_money - tm_unlevered

# ---- broadcast to both rows ----
deal_metrics = pd.DataFrame({
    "deal_id": d_equity.index,
    "d_equity": d_equity.values,
    "net_capital_gain": net_capital_gain.values,
    "invested_capital": invested_capital.values,
    "times_money": times_money.values,
    "tm_unlevered": tm_unlevered.values,
    "leverage_effect": leverage_effect.values,
}).set_index("deal_id")

out = df.drop(columns=["_ref_dt","_rank"]).merge(
    deal_metrics, left_on="deal_id", right_index=True, how="left"
)

# ---- persist + checks ----
out.to_csv(TARGET_CSV, index=False)

chk = pd.read_csv(TARGET_CSV, dtype={"deal_id": str})
# exactly 2 rows per surviving deal
assert chk.groupby("deal_id")["id"].nunique().eq(2).all(), "Row cardinality changed."
# no NaNs in core outputs
req = ["d_equity","net_capital_gain","invested_capital","times_money","tm_unlevered","leverage_effect"]
assert set(req).issubset(chk.columns), "Missing expected columns."
assert chk[req].notna().all().all(), "NaNs in deal-level outputs after merge."

print("Deal-level metrics added (exit-row div/cap applied):",
      ["d_equity","net_capital_gain","invested_capital","times_money","tm_unlevered","leverage_effect"])


Deal-level metrics added (exit-row div/cap applied): ['d_equity', 'net_capital_gain', 'invested_capital', 'times_money', 'tm_unlevered', 'leverage_effect']


In [139]:
# === Absolute Value Drivers: Multiple, FCF, EBITDA (PBA-aligned) ===

from pathlib import Path
import pandas as pd
import numpy as np

# -- helper --
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- CWD: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# -- load + guarantees --
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

# -- order within deal: 1 = entry (earliest), 2 = exit/latest --
df["_ref_dt"] = pd.to_datetime(df["reference_date"], errors="coerce")
df["_rank"] = df.groupby("deal_id")["_ref_dt"].rank(method="first", ascending=True)

def num(s): return pd.to_numeric(s, errors="coerce")

xe  = num(df["xebitda"])
eb  = num(df["ebitda"])
rv  = num(df["revenue"])
mg  = num(df["ebitda_margin"])
nd  = num(df["net_debt"])
div = num(df["dividends"])
cap = num(df["capital_injections"])

# -- pick entry / exit values per deal --
def pick(series, rank_val, name):
    s = series[df["_rank"] == rank_val]
    out = df.loc[df["_rank"] == rank_val, ["deal_id"]].assign(**{name: s.values})
    return out.groupby("deal_id")[name].first()

xebitda_entry  = pick(xe, 1, "xebitda_entry")
xebitda_exit   = pick(xe, 2, "xebitda_exit")
ebitda_entry   = pick(eb, 1, "ebitda_entry")
ebitda_exit    = pick(eb, 2, "ebitda_exit")
revenue_entry  = pick(rv, 1, "revenue_entry")
revenue_exit   = pick(rv, 2, "revenue_exit")
margin_entry   = pick(mg, 1, "margin_entry")
margin_exit    = pick(mg, 2, "margin_exit")
net_debt_entry = pick(nd, 1, "net_debt_entry")
net_debt_exit  = pick(nd, 2, "net_debt_exit")
div_exit       = pick(div, 2, "div_exit")
cap_exit       = pick(cap, 2, "cap_exit")

# -- driver sanity and required-data mask (drop undefined cores) --
xebitda_entry  = xebitda_entry.where(np.isfinite(xebitda_entry))
xebitda_exit   = xebitda_exit.where(np.isfinite(xebitda_exit))
margin_entry   = margin_entry.where(np.isfinite(margin_entry))

need = pd.Series(True, index=xebitda_entry.index)
need &= xebitda_entry.notna()
need &= xebitda_exit.notna()
need &= margin_entry.notna()
need &= ebitda_entry.notna() & ebitda_exit.notna()
need &= revenue_entry.notna() & revenue_exit.notna()
need &= net_debt_entry.notna() & net_debt_exit.notna()
need &= div_exit.notna() & cap_exit.notna()

if not need.all():
    dropped = int((~need).sum())
    print(f"Dropping {dropped} deal(s) due to undefined core drivers.")
    xebitda_entry  = xebitda_entry[need]
    xebitda_exit   = xebitda_exit[need]
    ebitda_entry   = ebitda_entry[need]
    ebitda_exit    = ebitda_exit[need]
    revenue_entry  = revenue_entry[need]
    revenue_exit   = revenue_exit[need]
    margin_entry   = margin_entry[need]
    margin_exit    = margin_exit[need]
    net_debt_entry = net_debt_entry[need]
    net_debt_exit  = net_debt_exit[need]
    div_exit       = div_exit[need]
    cap_exit       = cap_exit[need]
    survivor_deals = set(need[need].index)
    df = df[df["deal_id"].isin(survivor_deals)].copy()

# -- deltas (exit - entry); deleveraging defined as entry − exit (positive if debt reduced) --
d_multiple = xebitda_exit - xebitda_entry
d_ebitda   = ebitda_exit  - ebitda_entry
d_revenue  = revenue_exit - revenue_entry
d_margin   = margin_exit  - margin_entry
d_debt     = net_debt_entry - net_debt_exit

# -- effects (PBA absolute) --
multiple_effect = d_multiple * ebitda_entry
fcf_effect = d_debt + div_exit + cap_exit
multiple_ebitda_combination_effect = d_multiple * d_ebitda
ebitda_effect = d_ebitda * xebitda_entry
sales_effect = d_revenue * margin_entry * xebitda_entry
margin_effect = d_margin * revenue_entry * xebitda_entry
sales_margin_combination_effect = d_revenue * d_margin * xebitda_entry

# -- reconciliation to equity bridge (diagnostic residual) --
ev_entry = xebitda_entry * ebitda_entry
ev_exit  = xebitda_exit  * ebitda_exit
delta_ev = ev_exit - ev_entry

net_cap_gain_implied = delta_ev + (net_debt_entry - net_debt_exit) + div_exit + cap_exit
sum_effects = multiple_effect + ebitda_effect + multiple_ebitda_combination_effect + fcf_effect
effects_residual = sum_effects - net_cap_gain_implied
effects_residual_rel = effects_residual / net_cap_gain_implied.replace(0, np.nan).abs()

# -- bundle per-deal results and broadcast to both rows --
deal_effects = pd.DataFrame({
    "deal_id": d_multiple.index,
    "multiple_effect": multiple_effect.values,
    "fcf_effect": fcf_effect.values,
    "multiple_ebitda_combination_effect": multiple_ebitda_combination_effect.values,
    "ebitda_effect": ebitda_effect.values,
    "sales_effect": sales_effect.values,
    "margin_effect": margin_effect.values,
    "sales_margin_combination_effect": sales_margin_combination_effect.values,
    "effects_residual": effects_residual.values,
    "effects_residual_rel": effects_residual_rel.fillna(0.0).values,
}).set_index("deal_id")

out = df.drop(columns=["_ref_dt", "_rank"]).merge(
    deal_effects, left_on="deal_id", right_index=True, how="left"
)

# -- persist + checks --
out.to_csv(TARGET_CSV, index=False)

chk = pd.read_csv(TARGET_CSV, dtype={"deal_id": str})
assert chk.groupby("deal_id")["id"].nunique().eq(2).all(), "Row cardinality changed."
req = [
    "multiple_effect","fcf_effect","multiple_ebitda_combination_effect",
    "ebitda_effect","sales_effect","margin_effect","sales_margin_combination_effect",
    "effects_residual","effects_residual_rel"
]
assert set(req).issubset(chk.columns), "Missing effect columns."
assert chk[req].notna().all().all(), "NaNs in effects after merge."

bad_rec = (chk.groupby("deal_id")["effects_residual"].first().abs() > 1e-6) & \
          (chk.groupby("deal_id")["effects_residual_rel"].first().abs() > 1e-6)
if bad_rec.any():
    print(f"Warning: {int(bad_rec.sum())} deal(s) fail effect reconciliation (abs/rel tolerance).")

print("Deal-level effects added:",
      ["multiple_effect","fcf_effect","multiple_ebitda_combination_effect",
       "ebitda_effect","sales_effect","margin_effect","sales_margin_combination_effect",
       "effects_residual","effects_residual_rel"])


Deal-level effects added: ['multiple_effect', 'fcf_effect', 'multiple_ebitda_combination_effect', 'ebitda_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect', 'effects_residual', 'effects_residual_rel']


In [140]:
# === Relative Value Drivers: shares of net capital gain (deal-level, with residual) ===

from pathlib import Path
import pandas as pd
import numpy as np

def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        p = parent / rel_path
        if p.exists():
            return p.resolve()
    raise FileNotFoundError(f"Couldn't locate {rel_path} from {Path.cwd()}")

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + guarantees ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

# Effects computed earlier (absolute, per deal but broadcast to both rows)
effects = [
    "multiple_effect",
    "fcf_effect",
    "multiple_ebitda_combination_effect",
    "ebitda_effect",
    "sales_effect",
    "margin_effect",
    "sales_margin_combination_effect",
]
# include reconciliation residual from previous cell
effects_all = effects + ["effects_residual"]

# ---- compute relative shares (per deal), robust to tiny denominators ----
ncg = pd.to_numeric(df["net_capital_gain"], errors="coerce")
# Treat ultra-small |NCG| as zero to avoid exploding ratios
eps = 1e-12
valid_den = ncg.abs() > eps

for e in effects_all:
    num = pd.to_numeric(df[e], errors="coerce")
    rel = pd.Series(np.nan, index=df.index, dtype=float)
    rel[valid_den] = num[valid_den] / ncg[valid_den]
    df[f"relative_{e}"] = rel

# ---- per-deal sanity: sums should be ~1 (only where denom is valid) ----
grp = df.groupby("deal_id")
rel_sum = grp[[f"relative_{e}" for e in effects_all]].first().sum(axis=1)
ok = rel_sum[valid_den.groupby(df["deal_id"]).first()].sub(1.0).abs() <= 1e-6
if not ok.all():
    bad = (~ok).sum()
    print(f"Warning: {bad} deal(s) have relative effects that do not sum to 1 within tolerance.")

# ---- persist ----
df.to_csv(TARGET_CSV, index=False)
print("Computed relative effects (including residual) for:",
      effects_all)


Computed relative effects (including residual) for: ['multiple_effect', 'fcf_effect', 'multiple_ebitda_combination_effect', 'ebitda_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect', 'effects_residual']


In [141]:
# === QA + TM_unlevered-scaled (tmu_contrib_) driver contributions, incl. EBITDA sub-breakdown ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        cand = parent / rel_path
        if cand.exists():
            return cand.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels."
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + guarantee ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."
print("Deals in QA scope:", df["deal_id"].nunique())

num = lambda s: pd.to_numeric(s, errors="coerce")

# --- required columns
abs_main = ["multiple_effect","ebitda_effect","multiple_ebitda_combination_effect","fcf_effect"]
abs_sub  = ["sales_effect","margin_effect","sales_margin_combination_effect"]
rel_all  = [f"relative_{c}" for c in abs_main + abs_sub]

rel = df[rel_all].apply(num)
tm  = num(df["times_money"])
tmu = num(df["tm_unlevered"])
ic  = num(df["invested_capital"])

# ---- driver completeness mask (NaN/Inf guard); do not drop, just exclude from QA tallies
drivers = abs_main + abs_sub + ["times_money","tm_unlevered","invested_capital"]
drv_ok_row = df[drivers].apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan).notna().all(axis=1)
drivers_ok_deal = drv_ok_row.groupby(df["deal_id"]).all()

# ---------- (A) TM-scale (levered) check: (Σ abs main)/IC ≈ TM ----------
sum_abs_main = df[abs_main].apply(num).sum(axis=1, min_count=1)
tm_from_abs  = (sum_abs_main / ic).where(ic != 0)

diff_tm = tm - tm_from_abs
# 1% band with floor to avoid zero-tolerance near TM≈0
tol_tm  = np.maximum(1e-9, 0.01 * tm.abs())

ok_tm_row  = (diff_tm.abs() <= tol_tm) & np.isfinite(diff_tm) & np.isfinite(tol_tm)
ok_tm_deal = ok_tm_row.groupby(df["deal_id"]).all() & drivers_ok_deal
df["qa_tm_diff"] = diff_tm

# ---------- (B) TM_unlevered via relative shares: Σ (relative_main * tmu) ≈ tmu ----------
for c in abs_main + abs_sub:
    df[f"tmu_contrib_{c}"] = rel[f"relative_{c}"] * tmu

tmu_from_rel = df[[f"tmu_contrib_{c}" for c in abs_main]].sum(axis=1, min_count=1)
diff_tmu = tmu - tmu_from_rel
tol_tmu  = np.maximum(1e-9, 0.01 * tmu.abs())

ok_tmu_row  = (diff_tmu.abs() <= tol_tmu) & np.isfinite(diff_tmu) & np.isfinite(tol_tmu)
ok_tmu_deal = ok_tmu_row.groupby(df["deal_id"]).all() & drivers_ok_deal
df["qa_tmu_diff"] = diff_tmu

# ---------- (C) EBITDA sub-breakdown on TM_unlevered scale ----------
tmu_contrib_ebitda = df["tmu_contrib_ebitda_effect"]
tmu_contrib_subsum = df[[
    "tmu_contrib_sales_effect","tmu_contrib_margin_effect","tmu_contrib_sales_margin_combination_effect"
]].sum(axis=1, min_count=1)

diff_tmu_ebitda = tmu_contrib_ebitda - tmu_contrib_subsum
tol_tmu_ebitda  = np.maximum(1e-9, 0.01 * tmu_contrib_ebitda.abs())
ok_tmu_ebitda_row  = (diff_tmu_ebitda.abs() <= tol_tmu_ebitda) & np.isfinite(diff_tmu_ebitda) & np.isfinite(tol_tmu_ebitda)
ok_tmu_ebitda_deal = ok_tmu_ebitda_row.groupby(df["deal_id"]).all() & drivers_ok_deal
df["qa_tmu_ebitda_sub_diff"] = diff_tmu_ebitda

# ---------- Deal-level diffs (single value per deal), merged back for convenience ----------
deal_tm_diff          = diff_tm.groupby(df["deal_id"]).first()
deal_tmu_diff         = diff_tmu.groupby(df["deal_id"]).first()
deal_tmu_ebitda_diff  = diff_tmu_ebitda.groupby(df["deal_id"]).first()

df = df.merge(
    pd.DataFrame({
        "deal_id": deal_tm_diff.index,
        "qa_tm_diff_deal": deal_tm_diff.values,
        "qa_tmu_diff_deal": deal_tmu_diff.values,
        "qa_tmu_ebitda_sub_diff_deal": deal_tmu_ebitda_diff.values,
    }),
    on="deal_id", how="left"
)

# ---- persist ----
df.to_csv(TARGET_CSV, index=False)

# ---- summaries (deal-level) ----
def summarize(name, mask):
    # mask is a Series indexed by deal_id
    ok = int(mask.sum())
    out = int((~mask).sum())
    print(f"{name}: deals within ±1% = {ok} | outside ±1% = {out}")

summarize("TM (levered) identity", ok_tm_deal)
summarize("TM_unlevered (sum of tmu_contrib main effects)", ok_tmu_deal)
summarize("EBITDA sub-breakdown on TM_unlevered scale", ok_tmu_ebitda_deal)

print("tmu_contrib_ columns written for:", abs_main + abs_sub)


Deals in QA scope: 356
TM (levered) identity: deals within ±1% = 356 | outside ±1% = 0
TM_unlevered (sum of tmu_contrib main effects): deals within ±1% = 356 | outside ±1% = 0
EBITDA sub-breakdown on TM_unlevered scale: deals within ±1% = 356 | outside ±1% = 0
tmu_contrib_ columns written for: ['multiple_effect', 'ebitda_effect', 'multiple_ebitda_combination_effect', 'fcf_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect']
